<img src="https://docs.dask.org/en/latest/_images/dask_horizontal.svg" align="right" width="30%" alt="Dask logo">

# Bag: Parallel Lists for semi-structured data

Dask-bag excels in processing data that can be represented as a sequence of arbitrary inputs. We'll refer to this as "messy" data, because it can contain complex nested structures, missing fields, mixtures of data types, etc. The *functional* programming style fits very nicely with standard Python iteration, such as can be found in the `itertools` module.

Messy data is often encountered at the beginning of data processing pipelines when large volumes of raw data are first consumed. The initial set of data might be JSON, CSV, XML, or any other format that does not enforce strict structure and datatypes.
For this reason, the initial data massaging and processing is often done with Python `list`s, `dict`s, and `set`s.

These core data structures are optimized for general-purpose storage and processing.  Adding streaming computation with iterators/generator expressions or libraries like `itertools` or [`toolz`](https://toolz.readthedocs.io/en/latest/) let us process large volumes in a small space.  If we combine this with parallel processing then we can churn through a fair amount of data.

Dask.bag is a high-level Dask collection to automate common workloads of this form.  In a nutshell

    dask.bag = map, filter, toolz + parallel execution
    
**Related Documentation**

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Create data

In [1]:
%run prep.py -d accounts

## Setup

Again, we'll use the distributed scheduler. Schedulers will be explained in depth [later](05_distributed.ipynb).

In [2]:
from dask.distributed import Client

client = Client(n_workers=4)

## Creation

You can create a `Bag` from a Python sequence, from files, from data on S3, etc.
We demonstrate using `.take()` to show elements of the data. (Doing `.take(1)` results in a tuple with one element)

Note that the data are partitioned into blocks, and there are many items per block. In the first example, the two partitions contain five elements each, and in the following two, each file is partitioned into one or more bytes blocks.

In [3]:
# each element is an integer
import dask.bag as db
b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10], npartitions=2)
b.take(3)

(1, 2, 3)

In [4]:
# each element is a text file, where each line is a JSON object
# note that the compression is handled automatically
import os
b = db.read_text(os.path.join('data', 'accounts.*.json.gz'))
b.take(1)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 2843, "amount": 1469}, {"transaction-id": 3009, "amount": 1709}, {"transaction-id": 3715, "amount": 1572}, {"transaction-id": 4273, "amount": 1512}, {"transaction-id": 4403, "amount": 1658}, {"transaction-id": 5326, "amount": 1689}, {"transaction-id": 7121, "amount": 1446}, {"transaction-id": 8318, "amount": 1678}, {"transaction-id": 8599, "amount": 1768}, {"transaction-id": 9771, "amount": 1546}]}\n',)

In [5]:
# Edit sources.py to configure source locations
import sources
sources.bag_url

's3://dask-data/nyc-taxi/2015/yellow_tripdata_2015-01.csv'

In [6]:
# Requires `s3fs` library
# each partition is a remote CSV text file
b = db.read_text(sources.bag_url,
                 storage_options={'anon': True})
b.take(1)

('VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RateCodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n',)

## Manipulation

`Bag` objects hold the standard functional API found in projects like the Python standard library, `toolz`, or `pyspark`, including `map`, `filter`, `groupby`, etc..

Operations on `Bag` objects create new bags.  Call the `.compute()` method to trigger execution, as we saw for `Delayed` objects.  

In [7]:
def is_even(n):
    return n % 2 == 0

b = db.from_sequence([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
c = b.filter(is_even).map(lambda x: x ** 2)
c

dask.bag<lambda, npartitions=10>

In [8]:
# blocking form: wait for completion (which is very fast in this case)
c.compute()

[4, 16, 36, 64, 100]

### Example: Accounts JSON data

We've created a fake dataset of gzipped JSON data in your data directory.  This is like the example used in the `DataFrame` example we will see later, except that it has bundled up all of the entries for each individual `id` into a single record.  This is similar to data that you might collect off of a document store database or a web API.

Each line is a JSON encoded dictionary with the following keys

*  id: Unique identifier of the customer
*  name: Name of the customer
*  transactions: List of `transaction-id`, `amount` pairs, one for each transaction for the customer in that file

In [9]:
filename = os.path.join('data', 'accounts.*.json.gz')
lines = db.read_text(filename)
lines.take(3)

('{"id": 0, "name": "Jerry", "transactions": [{"transaction-id": 2843, "amount": 1469}, {"transaction-id": 3009, "amount": 1709}, {"transaction-id": 3715, "amount": 1572}, {"transaction-id": 4273, "amount": 1512}, {"transaction-id": 4403, "amount": 1658}, {"transaction-id": 5326, "amount": 1689}, {"transaction-id": 7121, "amount": 1446}, {"transaction-id": 8318, "amount": 1678}, {"transaction-id": 8599, "amount": 1768}, {"transaction-id": 9771, "amount": 1546}]}\n',
 '{"id": 1, "name": "Norbert", "transactions": [{"transaction-id": 28, "amount": 1016}, {"transaction-id": 212, "amount": 1019}, {"transaction-id": 280, "amount": 986}, {"transaction-id": 397, "amount": 1001}, {"transaction-id": 856, "amount": 1014}, {"transaction-id": 1167, "amount": 1030}, {"transaction-id": 1622, "amount": 1001}, {"transaction-id": 2325, "amount": 1051}, {"transaction-id": 2685, "amount": 981}, {"transaction-id": 2759, "amount": 1015}, {"transaction-id": 3103, "amount": 989}, {"transaction-id": 3329, "am

Our data comes out of the file as lines of text. Notice that file decompression happened automatically. We can make this data look more reasonable by mapping the `json.loads` function onto our bag.

In [10]:
import json
js = lines.map(json.loads)
# take: inspect first few elements
js.take(3)

({'id': 0,
  'name': 'Jerry',
  'transactions': [{'transaction-id': 2843, 'amount': 1469},
   {'transaction-id': 3009, 'amount': 1709},
   {'transaction-id': 3715, 'amount': 1572},
   {'transaction-id': 4273, 'amount': 1512},
   {'transaction-id': 4403, 'amount': 1658},
   {'transaction-id': 5326, 'amount': 1689},
   {'transaction-id': 7121, 'amount': 1446},
   {'transaction-id': 8318, 'amount': 1678},
   {'transaction-id': 8599, 'amount': 1768},
   {'transaction-id': 9771, 'amount': 1546}]},
 {'id': 1,
  'name': 'Norbert',
  'transactions': [{'transaction-id': 28, 'amount': 1016},
   {'transaction-id': 212, 'amount': 1019},
   {'transaction-id': 280, 'amount': 986},
   {'transaction-id': 397, 'amount': 1001},
   {'transaction-id': 856, 'amount': 1014},
   {'transaction-id': 1167, 'amount': 1030},
   {'transaction-id': 1622, 'amount': 1001},
   {'transaction-id': 2325, 'amount': 1051},
   {'transaction-id': 2685, 'amount': 981},
   {'transaction-id': 2759, 'amount': 1015},
   {'transac

### Basic Queries

Once we parse our JSON data into proper Python objects (`dict`s, `list`s, etc.) we can perform more interesting queries by creating small Python functions to run on our data.

In [11]:
# filter: keep only some elements of the sequence
js.filter(lambda record: record['name'] == 'Alice').take(5)

({'id': 15,
  'name': 'Alice',
  'transactions': [{'transaction-id': 123, 'amount': 798},
   {'transaction-id': 167, 'amount': 975},
   {'transaction-id': 408, 'amount': 740},
   {'transaction-id': 1452, 'amount': 676},
   {'transaction-id': 2001, 'amount': 785},
   {'transaction-id': 2145, 'amount': 790},
   {'transaction-id': 2500, 'amount': 877},
   {'transaction-id': 2767, 'amount': 706},
   {'transaction-id': 2836, 'amount': 634},
   {'transaction-id': 2979, 'amount': 728},
   {'transaction-id': 3032, 'amount': 832},
   {'transaction-id': 3327, 'amount': 809},
   {'transaction-id': 3664, 'amount': 682},
   {'transaction-id': 3954, 'amount': 685},
   {'transaction-id': 4168, 'amount': 771},
   {'transaction-id': 4200, 'amount': 600},
   {'transaction-id': 4451, 'amount': 789},
   {'transaction-id': 4559, 'amount': 880},
   {'transaction-id': 4676, 'amount': 830},
   {'transaction-id': 5044, 'amount': 764},
   {'transaction-id': 5125, 'amount': 855},
   {'transaction-id': 5339, 'amo

In [12]:
def count_transactions(d):
    return {'name': d['name'], 'count': len(d['transactions'])}

# map: apply a function to each element
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .take(5))

({'name': 'Alice', 'count': 37},
 {'name': 'Alice', 'count': 3},
 {'name': 'Alice', 'count': 51},
 {'name': 'Alice', 'count': 36},
 {'name': 'Alice', 'count': 10})

In [13]:
# pluck: select a field, as from a dictionary, element[field]
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .take(5))

(37, 3, 51, 36, 10)

In [14]:
# Average number of transactions for all of the Alice entries
(js.filter(lambda record: record['name'] == 'Alice')
   .map(count_transactions)
   .pluck('count')
   .mean()
   .compute())

32.19725557461407

### Use `flatten` to de-nest

In the example below we see the use of `.flatten()` to flatten results.  We compute the average amount for all transactions for all Alices.

In [15]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .take(3))

([{'transaction-id': 123, 'amount': 798},
  {'transaction-id': 167, 'amount': 975},
  {'transaction-id': 408, 'amount': 740},
  {'transaction-id': 1452, 'amount': 676},
  {'transaction-id': 2001, 'amount': 785},
  {'transaction-id': 2145, 'amount': 790},
  {'transaction-id': 2500, 'amount': 877},
  {'transaction-id': 2767, 'amount': 706},
  {'transaction-id': 2836, 'amount': 634},
  {'transaction-id': 2979, 'amount': 728},
  {'transaction-id': 3032, 'amount': 832},
  {'transaction-id': 3327, 'amount': 809},
  {'transaction-id': 3664, 'amount': 682},
  {'transaction-id': 3954, 'amount': 685},
  {'transaction-id': 4168, 'amount': 771},
  {'transaction-id': 4200, 'amount': 600},
  {'transaction-id': 4451, 'amount': 789},
  {'transaction-id': 4559, 'amount': 880},
  {'transaction-id': 4676, 'amount': 830},
  {'transaction-id': 5044, 'amount': 764},
  {'transaction-id': 5125, 'amount': 855},
  {'transaction-id': 5339, 'amount': 903},
  {'transaction-id': 5360, 'amount': 707},
  {'transactio

In [16]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .take(3))

({'transaction-id': 123, 'amount': 798},
 {'transaction-id': 167, 'amount': 975},
 {'transaction-id': 408, 'amount': 740})

In [17]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .take(3))

(798, 975, 740)

In [18]:
(js.filter(lambda record: record['name'] == 'Alice')
   .pluck('transactions')
   .flatten()
   .pluck('amount')
   .mean()
   .compute())

579.197432209259

### Groupby and Foldby

Often we want to group data by some function or key.  We can do this either with the `.groupby` method, which is straightforward but forces a full shuffle of the data (expensive) or with the harder-to-use but faster `.foldby` method, which does a streaming combined groupby and reduction.

*  `groupby`:  Shuffles data so that all items with the same key are in the same key-value pair
*  `foldby`:  Walks through the data accumulating a result per key

*Note: the full groupby is particularly bad. In actual workloads you would do well to use `foldby` or switch to `DataFrame`s if possible.*

### `groupby`

Groupby collects items in your collection so that all items with the same value under some function are collected together into a key-value pair.

In [19]:
b = db.from_sequence(['Alice', 'Bob', 'Charlie', 'Dan', 'Edith', 'Frank'])
b.groupby(len).compute()  # names grouped by length

[(7, ['Charlie']), (3, ['Bob', 'Dan']), (5, ['Alice', 'Edith', 'Frank'])]

In [20]:
b = db.from_sequence(list(range(10)))
b.groupby(lambda x: x % 2).compute()

[(0, [0, 2, 4, 6, 8]), (1, [1, 3, 5, 7, 9])]

In [21]:
b.groupby(lambda x: x % 2).starmap(lambda k, v: (k, max(v))).compute()

[(0, 8), (1, 9)]

### `foldby`

Foldby can be quite odd at first.  It is similar to the following functions from other libraries:

*  [`toolz.reduceby`](http://toolz.readthedocs.io/en/latest/streaming-analytics.html#streaming-split-apply-combine)
*  [`pyspark.RDD.combineByKey`](http://abshinn.github.io/python/apache-spark/2014/10/11/using-combinebykey-in-apache-spark/)

When using `foldby` you provide 

1.  A key function on which to group elements
2.  A binary operator such as you would pass to `reduce` that you use to perform reduction per each group
3.  A combine binary operator that can combine the results of two `reduce` calls on different parts of your dataset.

Your reduction must be associative.  It will happen in parallel in each of the partitions of your dataset.  Then all of these intermediate results will be combined by the `combine` binary operator.

In [22]:
b.foldby(lambda x: x % 2, binop=max, combine=max).compute()

[(0, 8), (1, 9)]

### Example with account data

We find the number of people with the same name.

In [23]:
%%time
# Warning, this one takes a while...
result = js.groupby(lambda item: item['name']).starmap(lambda k, v: (k, len(v))).compute()
print(sorted(result))

[('Alice', 137), ('Alice', 141), ('Alice', 151), ('Alice', 154), ('Bob', 120), ('Bob', 120), ('Bob', 130), ('Bob', 130), ('Charlie', 203), ('Charlie', 203), ('Charlie', 219), ('Charlie', 221), ('Dan', 85), ('Dan', 86), ('Dan', 186), ('Edith', 132), ('Edith', 132), ('Edith', 142), ('Edith', 143), ('Frank', 120), ('Frank', 120), ('Frank', 130), ('Frank', 130), ('George', 108), ('George', 108), ('George', 116), ('George', 117), ('Hannah', 60), ('Hannah', 60), ('Hannah', 65), ('Hannah', 65), ('Ingrid', 144), ('Ingrid', 144), ('Ingrid', 156), ('Ingrid', 156), ('Jerry', 84), ('Jerry', 84), ('Jerry', 91), ('Jerry', 91), ('Kevin', 148), ('Kevin', 150), ('Kevin', 157), ('Kevin', 158), ('Laura', 132), ('Laura', 132), ('Laura', 142), ('Laura', 142), ('Michael', 96), ('Michael', 96), ('Michael', 104), ('Michael', 104), ('Norbert', 125), ('Norbert', 128), ('Norbert', 141), ('Norbert', 142), ('Oliver', 102), ('Oliver', 107), ('Oliver', 111), ('Oliver', 113), ('Patricia', 72), ('Patricia', 72), ('Pat

In [24]:
%%time
# This one is comparatively fast and produces the same result.
from operator import add
def incr(tot, _):
    return tot + 1

result = js.foldby(key='name', 
                   binop=incr, 
                   initial=0, 
                   combine=add, 
                   combine_initial=0).compute()
print(sorted(result))

[('Alice', 583), ('Bob', 500), ('Charlie', 846), ('Dan', 357), ('Edith', 549), ('Frank', 500), ('George', 449), ('Hannah', 250), ('Ingrid', 600), ('Jerry', 350), ('Kevin', 613), ('Laura', 548), ('Michael', 400), ('Norbert', 536), ('Oliver', 433), ('Patricia', 300), ('Quinn', 202), ('Ray', 321), ('Sarah', 320), ('Tim', 500), ('Ursula', 599), ('Victor', 500), ('Wendy', 399), ('Xavier', 400), ('Yvonne', 840), ('Zelda', 229)]
CPU times: user 138 ms, sys: 13.5 ms, total: 152 ms
Wall time: 485 ms


### Exercise: compute total amount per name

We want to groupby (or foldby) the `name` key, then add up the all of the amounts for each name.

Steps

1.  Create a small function that, given a dictionary like 

        {'name': 'Alice', 'transactions': [{'amount': 1, 'id': 123}, {'amount': 2, 'id': 456}]}
        
    produces the sum of the amounts, e.g. `3`
    
2.  Slightly change the binary operator of the `foldby` example above so that the binary operator doesn't count the number of entries, but instead accumulates the sum of the amounts.

In [25]:
# Your code here...

## DataFrames

For the same reasons that Pandas is often faster than pure Python, `dask.dataframe` can be faster than `dask.bag`.  We will work more with DataFrames later, but from the point of view of a Bag, it is frequently the end-point of the "messy" part of data ingestion—once the data can be made into a data-frame, then complex split-apply-combine logic will become much more straight-forward and efficient.

You can transform a bag with a simple tuple or flat dictionary structure into a `dask.dataframe` with the `to_dataframe` method.

In [26]:
df1 = js.to_dataframe()
df1.head()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)


,id,name,transactions
0,0,Jerry,"[{'transaction-id': 2843, 'amount': 1469}, {'t..."
1,1,Norbert,"[{'transaction-id': 28, 'amount': 1016}, {'tra..."
2,2,Wendy,"[{'transaction-id': 30, 'amount': 256}, {'tran..."
3,3,Edith,"[{'transaction-id': 233, 'amount': -238}, {'tr..."
4,4,Kevin,"[{'transaction-id': 130, 'amount': 175}, {'tra..."


This now looks like a well-defined DataFrame, and we can apply Pandas-like computations to it efficiently.

Using a Dask DataFrame, how long does it take to do our prior computation of numbers of people with the same name?  It turns out that `dask.dataframe.groupby()` beats `dask.bag.groupby()` by more than an order of magnitude; but it still cannot match `dask.bag.foldby()` for this case.

In [27]:
%time df1.groupby('name').id.count().compute().head()

CPU times: user 281 ms, sys: 8.77 ms, total: 289 ms
Wall time: 1.49 s


name
Alice      583
Bob        500
Charlie    846
Dan        357
Edith      549
Name: id, dtype: int64

### Denormalization

This DataFrame format is less-than-optimal because the `transactions` column is filled with nested data so Pandas has to revert to `object` dtype, which is quite slow in Pandas.  Ideally we want to transform to a dataframe only after we have flattened our data so that each record is a single `int`, `string`, `float`, etc..

In [28]:
def denormalize(record):
    # returns a list for each person, one item per transaction
    return [{'id': record['id'], 
             'name': record['name'], 
             'amount': transaction['amount'], 
             'transaction-id': transaction['transaction-id']}
            for transaction in record['transactions']]

transactions = js.map(denormalize).flatten()
transactions.take(3)

({'id': 0, 'name': 'Jerry', 'amount': 1469, 'transaction-id': 2843},
 {'id': 0, 'name': 'Jerry', 'amount': 1709, 'transaction-id': 3009},
 {'id': 0, 'name': 'Jerry', 'amount': 1572, 'transaction-id': 3715})

In [29]:
df = transactions.to_dataframe()
df.head()

,id,name,amount,transaction-id
0,0,Jerry,1469,2843
1,0,Jerry,1709,3009
2,0,Jerry,1572,3715
3,0,Jerry,1512,4273
4,0,Jerry,1658,4403


In [30]:
%%time
# number of transactions per name
# note that the time here includes the data load and ingestion
df.groupby('name')['transaction-id'].count().compute()

CPU times: user 199 ms, sys: 34.8 ms, total: 234 ms
Wall time: 1.23 s


name
Alice       18771
Bob         20424
Charlie     36223
Dan         15068
Edith       19328
Frank       21743
George      22539
Hannah      10396
Ingrid      29793
Jerry       10962
Kevin       30668
Laura       19908
Michael     16322
Norbert     16026
Oliver      11171
Patricia    11438
Quinn       10816
Ray          4977
Sarah        8219
Tim         32007
Ursula      33689
Victor      18063
Wendy       18623
Xavier      22430
Yvonne      29730
Zelda       10666
Name: transaction-id, dtype: int64

## Limitations

Bags provide very general computation (any Python function.)  This generality
comes at cost.  Bags have the following known limitations

1.  Bag operations tend to be slower than array/dataframe computations in the
    same way that Python tends to be slower than NumPy/Pandas
2.  ``Bag.groupby`` is slow.  You should try to use ``Bag.foldby`` if possible.
    Using ``Bag.foldby`` requires more thought. Even better, consider creating
    a normalised dataframe.

## Learn More

* [Bag documentation](https://docs.dask.org/en/latest/bag.html)
* [Bag screencast](https://youtu.be/-qIiJ1XtSv0)
* [Bag API](https://docs.dask.org/en/latest/bag-api.html)
* [Bag examples](https://examples.dask.org/bag.html)

## Shutdown

In [31]:
client.shutdown()

/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/miniconda3/envs/dask-tutorial/lib/python3.8/site-packages/dask/dataframe/backends.py:187: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  _numeric_index_types = (pd.Int64Index, pd.Float64Index, pd.UInt64Index)
/usr/share/mi